# DA- VAE, Diffusion (HRV past dataset)

* Using Keras/Tensorflow

In [ ]:
import os
import time
# import shap ## for XAI
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
# import pingouin as pg
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

In [ ]:
from numpy import dot
from numpy.linalg import norm

from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten
from keras.layers import Dense , Activation, Dropout, BatchNormalization
from keras.optimizers import Adam ,RMSprop
from keras.callbacks import Callback

from scipy.special import rel_entr
from scipy.stats import gaussian_kde
from scipy.spatial.distance import jensenshannon

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, train_test_split, ParameterGrid
from sklearn import decomposition, metrics
from sklearn.cluster import KMeans
from sklearn.metrics import cohen_kappa_score,f1_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import Normalizer, MinMaxScaler, RobustScaler
from sklearn.manifold import TSNE

from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Concatenate, Dense, Lambda, Conv1D, Flatten, Reshape, UpSampling1D, MaxPooling1D, concatenate, Layer
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.losses import mse, MeanSquaredError

***

---

## Original data check

In [ ]:
### In this research, the dataset is breast cancer dataset (tabular format)

data_ori = pd.read_csv('E:/RESEARCH/Datasets/HRV/HRV_samsung/HRV_REV_processed.csv')

In [ ]:
### data shape, variables check
print("The shape of the HRV dataset is:",data_ori.shape)
# print(public.columns)
data_ori.head()

In [ ]:
# data_ori.columns

In [ ]:
# 결측값이 하나라도 있는 행의 개수 확인
num_missing_rows = data_ori.isna().any(axis=1).sum()

print(f"The number of rows that contains at least one missing value: {num_missing_rows}")

In [ ]:
data_ori.columns

> Target value is "disorder" variable. \
> 1: major depressive disorder, 2: panic disorder, 3: control \
> -1 on each values, before ann/dnn classification

In [ ]:
# data_ori['disorder_mdd'].value_counts()
data_ori['disorder'] = data_ori['disorder']-1
data_ori['disorder'].value_counts()

In [ ]:
data_ori['disorder_mdd'].value_counts()

### Relabeling

In [ ]:
data_ori['disorder'] = data_ori['disorder'].replace({0: 'mdd'})
data_ori['disorder'] = data_ori['disorder'].replace({1: 'pd'})
data_ori['disorder'] = data_ori['disorder'].replace({2: 'con'})

### Variables drop

In [ ]:
info_list = ['sub', 'VISIT', 'age', 'gender', 'disorder', 'disorder_mdd']
scale_list = ['HAMD', 'HAMA', 'PDSS', 'ASI', 'APPQ', 'PSWQ', 'SPI', 'PSS', 'BIS', 'SSI']
drop_list = info_list + scale_list

In [ ]:
x = data_ori.drop(drop_list, axis=1)
# y = data_ori.disorder_mdd
y = data_ori.disorder

In [ ]:
y.value_counts()

In [ ]:
x.columns

In [ ]:
data_ori['disorder'].value_counts()

### Distribution check

In [ ]:
data_vis = data_ori.copy()
data_vis.head()

In [ ]:
info_ = ['sub', 'VISIT', 'age', 'gender', 'disorder_mdd']
# info_ = ['sub', 'VISIT', 'age', 'gender', 'disorder', 'disorder_mdd']
scale_ = ['HAMD', 'HAMA', 'PDSS', 'ASI', 'APPQ', 'PSWQ', 'SPI', 'PSS', 'BIS', 'SSI']
drop_ = info_ + scale_

In [ ]:
x_ = data_vis.drop(drop_, axis=1)

In [ ]:
x_.columns

#### KDE plot

In [ ]:
features = [col for col in list(x.columns)]

In [ ]:
def kdeplot_features(data, feature, title):
    
    df0 = data[data['disorder']=="mdd"]
    df1 = data[data['disorder']=="pd"]
    df2 = data[data['disorder']=="con"]

    df0 = df0.drop(['disorder'], axis=1)
    df1 = df1.drop(['disorder'], axis=1)
    df2 = df2.drop(['disorder'], axis=1)
    
    df0_values = df0[feature].to_numpy()
    df1_values = df1[feature].to_numpy()
    df2_values = df2[feature].to_numpy()
     
    plt.figure(figsize = (6, 2.5))
    
    sns.kdeplot(df0_values, color = 'y')
    sns.kdeplot(df1_values, color = 'b')
    sns.kdeplot(df2_values, color = 'g')
    
    plt.title(title, fontsize=15)
    plt.legend()
    plt.show();
    
    # del values_train , values_test
    # gc.collect()

In [ ]:
for var in features:
    kdeplot_features(x_, feature=var, title = var + "distribution")

#### JSD calculation

In [ ]:
## list to append distributional differences
variable = []
diff01 = []
diff02 = []
diff03 = []
diff12 = []
diff13 = []
diff23 = []

for feature in features:
    ## deleting any NA values in each feature column, for distance calculation
    data = data_vis.dropna(subset=[feature])

    ## split the dataset into each classes
    df0 = data[data['target_info']=="control"]
    df1 = data[data['target_info']=="mdd"]
    df2 = data[data['target_info']=="bpi"]
    df3 = data[data['target_info']=="bpii"]

    df0 = df0.drop(['target_info'], axis=1)
    df1 = df1.drop(['target_info'], axis=1)
    df2 = df2.drop(['target_info'], axis=1)
    df3 = df3.drop(['target_info'], axis=1)
    
    df0_values = df0[feature]
    df1_values = df1[feature]
    df2_values = df2[feature]
    df3_values = df3[feature]

    ## sampling based on the minimum size of the target info.
    sample_size = (min(df0_values.shape[0], df1_values.shape[0], df2_values.shape[0], df3_values.shape[0]))
    df0_sample = df0_values.sample(n=(sample_size))
    df1_sample = df1_values.sample(n=(sample_size))
    df2_sample = df2_values.sample(n=(sample_size))
    df3_sample = df3_values.sample(n=(sample_size))

    ## jensen-shannon divergence calculation and append
    variable.append(feature)
    diff01.append(jensenshannon(df0_sample, df1_sample))
    diff02.append(jensenshannon(df0_sample, df2_sample))
    diff03.append(jensenshannon(df0_sample, df3_sample))
    diff12.append(jensenshannon(df1_sample, df2_sample))
    diff13.append(jensenshannon(df1_sample, df3_sample))
    diff23.append(jensenshannon(df2_sample, df3_sample)) 

--------------

----------

## VAE model

In [ ]:
data_ori["disorder"].value_counts()

In [ ]:
data = data_ori.copy()

In [ ]:
data_con = data[data['disorder']=="con"]
data_pd = data[data['disorder']=="pd"]
data_mdd = data[data['disorder']=="mdd"]

In [ ]:
class Args:
    # arugments
    epochs=100
    bs=32
    lr=0.0001
    momentum=0.9
    num_classes= 2
    latent_dim = 16
    inter_dim1 = 32
    inter_dim2 = 16
    seed=710674

args = Args()

# np.random.seed(args.seed)
# random.seed(args.seed)
# torch.manual_seed(args.seed)

In [ ]:
scale_list = ['HAMD', 'HAMA', 'PDSS', 'ASI', 'APPQ', 'PSWQ', 'SPI', 'PSS', 'BIS', 'SSI']
etc_list = ['sub', 'VISIT', 'disorder_mdd']
demo_list = ['age', 'gender', 'disorder']

In [ ]:
x = data.drop((scale_list + etc_list + demo_list), axis=1)

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)

x[:] = (scaler.fit_transform(x[:])).round(decimals=6)
y = data.disorder  # mdd / pd / con

In [ ]:
data_x = x.copy()
data_x = data_x.fillna(data_x.mean())

In [ ]:
y.value_counts()

In [ ]:
### breast cancer wisconsin ####### BINARY CLASSIFICATION ##########

label = y
label = label.replace({'mdd': 0})
label = label.replace({'pd': 1})
label = label.replace({'con' : 2})


data_y = to_categorical(label, 3) ## into the format of one-hot encoding

In [ ]:
print("The size of x dataset is:", data_x.shape)
print("The size of y dataset is:", data_y.shape)

***

### Overall VAE data synthesizing model

VAE class explanations
* __init__: VAE 모델의 초기화 및 구성 요소를 정의합니다. 입력 차원 (input_dim)과 잠재 공간 차원 (latent_dim)을 파라미터로 받습니다.
* build_model: 인코더, 디코더, VAE 모델을 정의하고, VAE 손실을 설정합니다.
* train: 데이터를 받아 VAE 모델을 학습합니다.
* encode: 데이터를 잠재 공간으로 인코딩합니다.
* visualize_latent_space: t-SNE를 사용하여 잠재 공간을 시각화합니다.

In [ ]:
class VAE:
    def __init__(self, input_dim, latent_dim):
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.encoder = None
        self.decoder = None
        self.vae = None
        self.build_model()

    ## VAE model for data augmentation
    def build_model(self):
        ####################################################
        ## defining encoder section
        inputs = Input(shape=(self.input_dim,))
        
        # shallow model use
        h = Dense(32, activation='relu')(inputs)

        # # deeper model use
        # h = Dense(128, activation='relu')(inputs)
        # h = Dense(64, activation='relu')(h)
        # h = Dense(32, activation='relu')(h)

        # calculating mean/var
        z_mean = Dense(self.latent_dim)(h)
        z_log_var = Dense(self.latent_dim)(h)

        ## sampling section (for gaussian distribution)
        def sampling(args):
            z_mean, z_log_var = args
            batch = K.shape(z_mean)[0]
            dim = K.int_shape(z_mean)[1]
            epsilon = K.random_normal(shape=(batch, dim))
            return z_mean + K.exp(0.5 * z_log_var) * epsilon

        z = Lambda(sampling, output_shape=(self.latent_dim,))([z_mean, z_log_var])

        ####################################################
        ## defining decoder section

        # shallower model use
        decoder_h = Dense(32, activation='relu')
        decoder_mean = Dense(input_dim, activation='sigmoid')
        h_decoded = decoder_h(z)
        x_decoded_mean = decoder_mean(h_decoded)
        
        # # deeper model use
        # decoder_h1 = Dense(32, activation='relu')
        # decoder_h2 = Dense(64, activation='relu')
        # decoder_h3 = Dense(128, activation='relu')
        # decoder_mean = Dense(self.input_dim, activation='sigmoid')

        # h_decoded = decoder_h1(z)
        # h_decoded = decoder_h2(h_decoded)
        # h_decoded = decoder_h3(h_decoded)
        # x_decoded_mean = decoder_mean(h_decoded)

        ####################################################
        ## defining VAE model
        self.vae = Model(inputs, x_decoded_mean)

        ####################################################
        ## defining loss function (reconstruction + KL divergence)
        reconstruction_loss = MeanSquaredError()(inputs, x_decoded_mean)
        kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var + K.epsilon()), axis=-1)
        vae_loss = K.mean(reconstruction_loss + kl_loss)
        self.vae.add_loss(vae_loss)

        ####################################################
        ## compiling model
        self.vae.compile(optimizer=Adam(learning_rate=0.001))

        ####################################################
        ## Extracting encoder and decoder separately
        ## encoder section
        self.encoder = Model(inputs, z_mean)

        ## decoder section
        decoder_input = Input(shape=(self.latent_dim,))
        _h_decoded = decoder_h(decoder_input)
        _x_decoded_mean = decoder_mean(_h_decoded)
        self.decoder = Model(decoder_input, _x_decoded_mean)
        
        # #### deeper model
        # decoder_input = Input(shape=(self.latent_dim,))
        # _h_decoded = decoder_h1(decoder_input)
        # _h_decoded = decoder_h2(_h_decoded)
        # _h_decoded = decoder_h3(_h_decoded)
        # _x_decoded_mean = decoder_mean(_h_decoded)
        # self.decoder = Model(decoder_input, _x_decoded_mean)
    
    ####################################################
    ####################################################
    ## Model training
    def train(self, data, epochs, batch_size, validation_split):
        self.vae.fit(data, epochs=epochs, batch_size=batch_size, validation_split=validation_split)

    def encode(self, data):
        return self.encoder.predict(data)

    def decode(self, latent_points):
        return self.decoder.predict(latent_points)

    def generate_synthetic_data(self, num_samples=1):
        latent_samples = np.random.normal(size=(num_samples, self.latent_dim)) ## sampling from latent space
        return self.decode(latent_samples) ## synthetic data generation with decoder section

    def visualize_latent_space(self, data, labels):
        ## encode the dataset into latent space
        encoded_data = self.encode(data)

        ## latent space visualization with t-SNE
        tsne = TSNE(n_components=2, random_state=710674)
        encoded_data_tsne = tsne.fit_transform(encoded_data)

        ## Visualize
        plt.figure(figsize=(8, 6))
        plt.scatter(encoded_data_tsne[:, 0], encoded_data_tsne[:, 1], c=labels, cmap='viridis')
        plt.colorbar()
        plt.xlabel("t-SNE component 1")
        plt.ylabel("t-SNE component 2")
        plt.title("t-SNE visualization of the latent space")
        plt.show()

In [ ]:
vae = VAE(input_dim=data_x.shape[1], latent_dim=2)
vae.train(data_x, epochs = args.epochs, batch_size = args.bs, validation_split=0.2)

### Latent space visualization

In [ ]:
vae.visualize_latent_space(data_x, label)

### Synthetic data generation

* For mdd class augmentation

In [ ]:
synthetic_mdd = vae.generate_synthetic_data(num_samples=10)
gen_mdd = synthetic_mdd.copy()
gen_mdd = pd.DataFrame(gen_mdd, columns = data_x.columns)

* For pd class augmentation

In [ ]:
synthetic_pd = vae.generate_synthetic_data(num_samples=10)
gen_pd = synthetic_pd.copy()
gen_pd = pd.DataFrame(gen_pd, columns = data_x.columns)

* For con class augmentation

In [ ]:
synthetic_con = vae.generate_synthetic_data(num_samples=10)
gen_con = synthetic_con.copy()
gen_con = pd.DataFrame(gen_con, columns = data_x.columns)

In [ ]:
# # Synthetic data 생성
# synthetic_data = vae.generate_synthetic_data(num_samples=5)
# print("Generated synthetic data:")
# print(synthetic_data)

***

***

## Conditional VAE model

In [ ]:
data_cvae = data_ori.copy()

In [ ]:
data_cvae.shape

In [ ]:
scale_list = ['HAMD', 'HAMA', 'PDSS', 'ASI', 'APPQ', 'PSWQ', 'SPI', 'PSS', 'BIS', 'SSI']
etc_list = ['sub', 'VISIT', 'disorder_mdd']
demo_list = ['age', 'gender', 'disorder']

In [ ]:
data_cvae = data_cvae.drop((scale_list + etc_list), axis=1)

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)

data_cvae[:] = scaler.fit_transform(data_cvae[:])
data_cvae = data_cvae.round(decimals=5)

In [ ]:
data_cvae.columns

In [ ]:
data_cvae.head()

In [ ]:
# # y includes our labels and x includes our features
# y = data_cvae.diagnosis  # M or B 
# x = data_cvae.drop(demo_list, axis = 1 )

In [ ]:
demographics = data_cvae.loc[:, ('disorder', 'age', 'gender')]

In [ ]:
demographics.shape

In [ ]:
bio_signals = data_cvae.drop(demo_list, axis=1)

In [ ]:
bio_signals.shape

In [ ]:
demographics['disorder'].value_counts()

In [ ]:
# # 데이터에 NaN이나 무한대 값이 있는지 확인
# print(np.isnan(bio_signals).sum(), np.isinf(bio_signals).sum())
# print(np.isnan(demographics).sum(), np.isinf(demographics).sum())

In [ ]:
input_dim = bio_signals.shape[1]
condition_dim = demographics.shape[1]
latent_dim = 8

In [ ]:
# 입력 텐서
x_input = layers.Input(shape=(input_dim,), name='input')
c_input = layers.Input(shape=(condition_dim,), name='condition')

In [ ]:
# 조건부 입력과 원본 입력을 합침
combined_input = layers.Concatenate()([x_input, c_input])

# 인코더 네트워크
h = layers.Dense(64, activation='relu')(combined_input)
h = layers.Dense(32, activation='relu')(h)

# 잠재 공간의 평균과 로그 분산을 출력
z_mean = layers.Dense(latent_dim, name='z_mean')(h)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(h)

# 리파라메터라이제이션 트릭
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# 잠재 벡터 z
z = layers.Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

In [ ]:
# 잠재 벡터 z와 조건부 입력을 다시 합침
z_cond = layers.Concatenate()([z, c_input])

# 디코더 네트워크
decoder_h = layers.Dense(32, activation='relu')(z_cond)
decoder_h = layers.Dense(64, activation='relu')(decoder_h)
x_decoded = layers.Dense(input_dim, activation='sigmoid', name='output')(decoder_h)

In [ ]:
# CVAE 모델 (인코더와 디코더를 포함)
cvae = Model([x_input, c_input], x_decoded)

# VAE loss 계산
reconstruction_loss = mse(x_input, x_decoded)
reconstruction_loss *= input_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)

cvae.add_loss(vae_loss)
cvae.compile(optimizer='adam')
cvae.summary()

In [ ]:
# 모델 학습
cvae.fit([bio_signals, demographics], epochs=50, batch_size=32)

***

### Overall CVAE code

클래스 설명:
* __init__: CVAE 클래스는 초기화 시에 인코더, 디코더, 전체 CVAE 모델을 생성합니다.
* build_encoder: 인코더를 정의하며, 입력 데이터와 조건부 변수를 받아 잠재 변수 z를 생성합니다.
* build_decoder: 디코더를 정의하며, 잠재 변수 z와 조건부 변수를 받아 데이터를 재구성합니다.
* build_cvae: 인코더와 디코더를 결합하여 전체 CVAE 모델을 생성하고, 손실 함수를 정의합니다.
* train: 주어진 데이터를 사용하여 CVAE 모델을 학습시킵니다.
* generate_synthetic_data: 학습된 모델을 사용하여 주어진 조건부 변수에 따른 새로운 synthetic 데이터를 생성합니다.

In [ ]:
class CVAE:
    def __init__(self, input_dim, condition_dim, latent_dim):
        ## initializing data demensionality
        self.input_dim = input_dim
        self.condition_dim = condition_dim
        self.latent_dim = latent_dim
        
        ## generating encoder and decoder section
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()
        self.cvae = self.build_cvae()
    
    ## defining encoder function
    def build_encoder(self):
        x_input = layers.Input(shape=(self.input_dim,), name='input')
        c_input = layers.Input(shape=(self.condition_dim,), name='condition')
        
        combined_input = layers.Concatenate()([x_input, c_input])
        
        h = layers.Dense(64, activation='relu')(combined_input)
        h = layers.Dense(32, activation='relu')(h)
        
        z_mean = layers.Dense(self.latent_dim, name='z_mean')(h)
        z_log_var = layers.Dense(self.latent_dim, name='z_log_var')(h)
        
        ## sampling (reparameterization)
        def sampling(args):
            z_mean, z_log_var = args
            batch = K.shape(z_mean)[0]
            dim = K.int_shape(z_mean)[1]
            epsilon = K.random_normal(shape=(batch, dim))
            return z_mean + K.exp(0.5 * z_log_var) * epsilon
        
        z = layers.Lambda(sampling, output_shape=(self.latent_dim,), name='z')([z_mean, z_log_var])
        
        return Model([x_input, c_input], [z_mean, z_log_var, z], name='encoder')
    
    ## defining decoder function
    def build_decoder(self):
        z_input = layers.Input(shape=(self.latent_dim,), name='z_sampling')
        c_input = layers.Input(shape=(self.condition_dim,), name='condition')
        
        combined_input = layers.Concatenate()([z_input, c_input])
        
        h = layers.Dense(32, activation='relu')(combined_input)
        h = layers.Dense(64, activation='relu')(h)
        x_decoded = layers.Dense(self.input_dim, activation='sigmoid', name='output')(h)
        
        return Model([z_input, c_input], x_decoded, name='decoder')
    
    ## build and compile the CVAE model
    def build_cvae(self):
        x_input = layers.Input(shape=(self.input_dim,), name='input')
        c_input = layers.Input(shape=(self.condition_dim,), name='condition')
        
        z_mean, z_log_var, z = self.encoder([x_input, c_input])
        x_decoded = self.decoder([z, c_input])
        
        cvae = Model([x_input, c_input], x_decoded, name='cvae')
        
        reconstruction_loss = mse(x_input, x_decoded)
        reconstruction_loss *= self.input_dim
        
        kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        
        cvae_loss = K.mean(reconstruction_loss + kl_loss)
        cvae.add_loss(cvae_loss)
        cvae.compile(optimizer='adam')
        
        return cvae
    
    ## model training
    def train(self, x_train, c_train, epochs=50, batch_size=32):
        self.cvae.fit([x_train, c_train], epochs=epochs, batch_size=batch_size)
    
    ## synthetic data generation (for data augmentation)
    def generate_synthetic_data(self, condition, n_samples=1):
        z_samples = np.random.normal(size=(n_samples, self.latent_dim))
        synthetic_data = self.decoder.predict([z_samples, condition])
        return synthetic_data

In [ ]:
## defining datadimensionality
input_dim = bio_signals.shape[1]
condition_dim = demographics.shape[1]
latent_dim = 2

# CVAE 인스턴스 생성
cvae_model = CVAE(input_dim, condition_dim, latent_dim)

# 모델 학습
cvae.fit([bio_signals, demographics], epochs=50, batch_size=32)

In [ ]:
demographics.head()

In [ ]:
# 새로운 조건부 변수 설정 (예: 모든 값이 0.5인 경우)
new_condition = np.array([[0.5, 0.327, 1]])

# synthetic data 생성
synthetic_data = cvae_model.generate_synthetic_data(new_condition, n_samples=1)
print(synthetic_data)

----------

-----------

## Latent Diffusion Model

### encoder, decoder networks

In [ ]:
data = data_ori.copy()

In [ ]:
data_B = data[data['diagnosis']=="B"]
data_M = data[data['diagnosis']=="M"]

In [ ]:
# data_diffusion = data_ori.copy()
# data_diffusion = data_B.copy()
data_diffusion = data_M.copy()

In [ ]:
data_diffusion.shape

In [ ]:
# y includes our labels and x includes our features
list = ['Unnamed: 32','id','diagnosis']
x = data_diffusion.drop(list, axis = 1 )
y = data_diffusion.diagnosis  # M or B 

In [ ]:
data_x = x.copy()

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)
data_x[:] = scaler.fit_transform(data_x[:])
data_x = data_x.round(decimals=5)

In [ ]:
### breast cancer wisconsin ####### BINARY CLASSIFICATION ##########
label = y
label = label.replace({'B':0})
label = label.replace({'M': 1})

data_y = to_categorical(label, 2) ## into the format of one-hot encoding

In [ ]:
print("The size of x dataset is:", data_x.shape)
print("The size of y dataset is:", data_y.shape)

In [ ]:
# Sampling function for the latent space
def sampling(repara):
    z_mean, z_log_var = repara
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Define encoder
def build_encoder(input_shape, latent_dim):
    inputs = Input(shape=input_shape)
    x = Conv1D(32, 3, activation='relu', padding='same')(inputs)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    z_mean = Dense(latent_dim)(x)
    z_log_var = Dense(latent_dim)(x)
    return Model(inputs, [z_mean, z_log_var], name='encoder')

# Define decoder
def build_decoder(output_shape, latent_dim):
    latent_inputs = Input(shape=(latent_dim,))
    x = Dense(128, activation='relu')(latent_inputs)
    x = Dense(np.prod(output_shape), activation='relu')(x)
    x = Reshape(output_shape)(x)
    outputs = Conv1D(1, 3, activation='sigmoid', padding='same')(x)
    return Model(latent_inputs, outputs, name='decoder')

In [ ]:
input_shape = (data_x.shape[1], 1)  # Example input shape
latent_dim = 16

In [ ]:
input_shape

In [ ]:
## Define the VAE
encoder = build_encoder(input_shape, latent_dim)
decoder = build_decoder(input_shape, latent_dim)

inputs = Input(shape=input_shape)
z_mean, z_log_var = encoder(inputs)
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
outputs = decoder(z)

vae = Model(inputs, outputs, name='vae')

# Define the VAE loss
reconstruction_loss = MeanSquaredError()(inputs, outputs)
kl_loss = -0.5 * tf.reduce_mean(
    z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae_loss = reconstruction_loss + kl_loss
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# vae.summary()

In [ ]:
# encoder.summary()

In [ ]:
# decoder.summary()

### model training

In [ ]:
# Train the VAE
vae.fit(data_x, epochs=args.epochs, validation_split=0.2, batch_size=args.bs, verbose=2)

### extracting encoder, decoder

In [ ]:
# # Extract the encoder and decoder models
# encoder_model = encoder
# decoder_model = decoder

In [ ]:
# # Example of using the encoder to get the latent representation
# example_data = np.random.rand(1, 356, 1)  # Example single data point
# z_mean, z_log_var = encoder_model.predict(example_data)
# z_sampled = sampling([z_mean, z_log_var])

# print("Latent representation (z_mean):", z_mean)
# print("Latent representation (z_log_var):", z_log_var)
# print("Sampled latent vector:", z_sampled)

# # Example of using the decoder to generate data from the latent space
# synthetic_data = decoder_model.predict(z_sampled)

# print("Generated synthetic data:", synthetic_data)

### Synthetic data generation

In [ ]:
## for synthetic data generation from latent diffusion's latent space
def latent_diffusion_data_generation(decoder, latent_dim, num_samples):
    # sample random vectors from a standard normal distribution
    random_vectors = np.random.normal(size=(num_samples, latent_dim))
    # generating synthetic dataset with decoder
    synthetic_data = decoder.predict(random_vectors)
    return synthetic_data

In [ ]:
### sampling randomly from latent space
b_samples = 212  # number of synthetic dataset to generate (generate size for gen_B)
m_samples = 357  # number of synthetic dataset to generate (generate size for gen_M)

### generating synthetic dataset for each class
# synthetic_data_B = latent_diffusion_data_generation(decoder, latent_dim, b_samples)
synthetic_data_M = latent_diffusion_data_generation(decoder, latent_dim, m_samples)

In [ ]:
## Reshape synthetic data for Sequential model input
# Ensure this matches the input shape of the Sequential model
# synthetic_data_B = synthetic_data_B.reshape((synthetic_data_B.shape[0]), (synthetic_data_B.shape[1]))
synthetic_data_M = synthetic_data_M.reshape((synthetic_data_M.shape[0]), (synthetic_data_M.shape[1]))

In [ ]:
synthetic_data_B

In [ ]:
# gen_B = synthetic_data_B.copy()
# gen_B = pd.DataFrame(gen_B, columns=data_x.columns)

#####################
gen_M = synthetic_data_M.copy()
gen_M = pd.DataFrame(gen_M, columns=data_x.columns)

---------

-----------

## DNN classification/prediction

### using original dataset only

In [ ]:
# x.head()
# y.head()

In [ ]:
# y.value_counts()

In [ ]:
ori_x = x.copy()

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)

ori_x[:] = scaler.fit_transform(ori_x[:])
ori_x = ori_x.round(decimals=5)

In [ ]:
########## 3-CLASS CLASSIFICATION ##########
label = y.copy()
label = label.replace({'normal': 0})
label = label.replace({'mild': 1})
label = label.replace({'mod_severe': 2})

ori_y = to_categorical(label, 3) ## into the format of one-hot encoding

In [ ]:
ori_x.shape

### using original + generated dataset

> we generate synthesized dataset using VAE from above. \
> adopted original datset: data_mdd, data_bpi, data_bpii \
> synthesized into: gen_mdd, gen_bpi, gen_bpii \
> gen_control is not generated since "control group" has biggest number.

* Set target information into newly generated dataset

In [ ]:
gen_B['diagnosis'] = "B"
gen_M['diagnosis'] = "M"

* Preparing original dataset

In [ ]:
data_B = data_B.drop(['Unnamed: 32','id'], axis=1)
data_M = data_M.drop(['Unnamed: 32','id'], axis=1)

* Concat the original datasets into one original dataframe

In [ ]:
ori_df_list = [data_B, data_M]
ori_df_concat = pd.concat(ori_df_list, ignore_index=True)

In [ ]:
ori_df_concat.shape

In [ ]:
ori_df_concat['diagnosis'].value_counts()

* Concat the generated datasets into one gen dataframe

In [ ]:
gen_df_list = [gen_B, gen_M]
gen_df_concat = pd.concat(gen_df_list, ignore_index=True)

In [ ]:
gen_df_concat['diagnosis'].value_counts()

* Preparing x and y data vectors

In [ ]:
### Using MDD,BP dataset
ori_x = ori_df_concat.drop(['diagnosis'], axis=1)
ori_y = ori_df_concat.loc[:,["diagnosis"]]

gen_x = gen_df_concat.drop(['diagnosis'], axis=1)
gen_y = gen_df_concat.loc[:,["diagnosis"]]

In [ ]:
ori_x = ori_x.fillna(ori_x.mean())

In [ ]:
label = ori_y
label = label.replace({'B':0})
label = label.replace({'M': 1})

y_ori = to_categorical(label, 2) ## into the format of one-hot encoding

In [ ]:
label_ = gen_y
label_ = label_.replace({'B':0})
label_ = label_.replace({'M': 1})

y_gen = to_categorical(label_, 2) ## into the format of one-hot encoding

* Separating the test dataset only from original dataframe

In [ ]:
x_trainset, x_test, y_trainset, y_test = train_test_split(ori_x, y_ori, test_size = 0.4, random_state = 710674)

* Then concat the generated dataset with training dataset

In [ ]:
x_train_concat = pd.concat([x_trainset, gen_x], ignore_index=True)
y_train_concat = np.concatenate([y_trainset, y_gen])

* Then separating the validation dataset from concat dataframe

In [ ]:
x_train, x_vali, y_train, y_vali = train_test_split(x_train_concat, y_train_concat, test_size = 0.2, random_state = 710674)

In [ ]:
# ori_x[ori_x.isna().any(axis=1)].shape

In [ ]:
# # fig, ax = plt.subplots()
# plt.figure(figsize = (10, 5))
# plt.xlabel('Generated MDD BP feature variables',fontsize=10)
# plt.ylabel('Feature value',fontsize=10)
# plt.boxplot(ori_x)
# plt.show()

***

### model training + test

In [ ]:
class Args:
    # arugments
    epochs=300
    bs=32
    lr=0.001
    momentum=0.9
    num_classes= 3
    seed=710674

args = Args()

#### normal DNN

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(ori_x, ori_y, test_size = 0.2, random_state = 710674)

In [ ]:
x_train.shape

In [ ]:
############## FOR balanced CLASSIFICATION ###############
# class_weight = {0:1, 1:1.3, 2: 1.43} ## con 194, pd 149, mdd 136
class_weight = {0:1, 1: 1.68, 2: 3.7}  ## normal 258, mild 153, mod_severe 68

In [ ]:
#### Generate the model
## this model is adequate for JSD_big differences variables use
model = Sequential()
model.add(Dense(64, input_dim = x_train.shape[1], activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(128, activation = 'relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
# model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(args.num_classes, activation = 'softmax'))

In [ ]:
def cyclic_learning_rate(epoch, base_lr=0.001, max_lr=0.006, step_size=2000):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * max(0, (1 - x))
    return lr

In [ ]:
def triangular_lr(epoch, base_lr=0.001, max_lr=0.006, step_size=2000):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * max(0, (1 - x))
    return lr

In [ ]:
def triangular2_lr(epoch, base_lr=0.001, max_lr=0.006, step_size=2000):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * max(0, (1 - x)) / (2 ** (cycle - 1))
    return lr

In [ ]:
def exp_range_lr(epoch, base_lr=0.001, max_lr=0.006, step_size=2000, gamma=0.99994):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * max(0, (1 - x)) * (gamma ** epoch)
    return lr

* optimization function, model compile, and model training

In [ ]:
opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-5, momentum = args.momentum)

In [ ]:
# LearningRateScheduler 설정
lr_scheduler = LearningRateScheduler(lambda epoch: cyclic_learning_rate(epoch, base_lr=0.01, max_lr=0.006, step_size=300))
# lr_scheduler = LearningRateScheduler(lambda epoch: exp_range_lr(epoch, base_lr=0.01, max_lr=0.006, step_size=300, gamma=0.99994))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.bs, verbose=2)   ## using no class weight
# history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.bs, verbose=2, class_weight = class_weight)  ## using class weight
history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.bs, callbacks=[lr_scheduler], class_weight = class_weight, verbose=2) ## using LR scheduler callback

***

In [ ]:
y_predict = model.predict(x_test)
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')
auc = roc_auc_score(y_test, model.predict(x_test, verbose=0), multi_class='ovr')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("The AUC score is:", round(auc, 4))
print("=============================================")

***

#### k-fold

In [ ]:
x_trainset, x_test, y_trainset, y_test = train_test_split(ori_x, ori_y, test_size = 0.4, random_state = 710674)

In [ ]:
x_train, x_vali, y_train, y_vali = train_test_split(x_trainset, y_trainset, test_size = 0.2, random_state = 710674)

In [ ]:
inputs = np.concatenate((x_train, x_vali), axis = 0)
targets = np.concatenate((y_train, y_vali), axis = 0)

In [ ]:
fold_num = 1
split_num = 5
opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-6, momentum = args.momentum)
kfold = KFold(n_splits = split_num, shuffle = True)

In [ ]:
############## FOR balanced CLASSIFICATION ###############
# class_weight = {0:1, 1:1.3, 2: 1.43} ## con 194, pd 149, mdd 136
class_weight = {0:1, 1: 1.68, 2: 3.7}  ## normal 258, mild 153, mod_severe 68

In [ ]:
acc_per_fold = []
loss_per_fold = []

In [ ]:
x_train.shape

In [ ]:
x_train.columns

In [ ]:
for train, test in kfold.split(inputs, targets):
    model = Sequential()
    model.add(Dense(128, input_dim = x_train.shape[1], activation = 'relu'))
    model.add(Dense(64, activation = 'relu'))
    model.add(BatchNormalization())
    # model.add(Dropout(0.5)) #drop out
    model.add(Dense(32, activation = 'relu'))
    # model.add(Dense(32, activation = 'relu'))
    model.add(Dense(args.num_classes, activation = 'softmax'))
    
    ## model compile
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    
    print('----------------------------------------')
    print(f'Training or fold {fold_num} ... ')
    
    ## fit data to model
    history = model.fit(inputs[train], targets[train], batch_size = args.bs, epochs = args.epochs, verbose = 0, class_weight = class_weight)
    
    ## generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test])
    print(f'Score for fold {fold_num}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    ## increasing fold number
    fold_num = fold_num + 1
    
    
    
## Summarizing the results
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'>> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'>>> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'>>> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
y_predict = model.predict(x_test)
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("=============================================")

***

#### 1D-Conv

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(ori_x, ori_y, test_size = 0.4, random_state = 710674)

In [ ]:
x_train.shape

In [ ]:
# Build the 1D CNN model
model = Sequential()
# 1st Convolutional Layer
model.add(Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=3))

# Flatten the output and add Dense layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(args.num_classes, activation='softmax'))

In [ ]:
opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-5, momentum = args.momentum)

In [ ]:
# LearningRateScheduler 설정
lr_scheduler = LearningRateScheduler(lambda epoch: cyclic_learning_rate(epoch, base_lr=0.01, max_lr=0.006, step_size=300))
# lr_scheduler = LearningRateScheduler(lambda epoch: exp_range_lr(epoch, base_lr=0.01, max_lr=0.006, step_size=300, gamma=0.99994))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.bs, verbose=2)   ## using no class weight
# history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.bs, verbose=2, class_weight = class_weight)  ## using class weight
history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.bs, callbacks=[lr_scheduler], class_weight = class_weight, verbose=2) ## using LR scheduler callback

***

In [ ]:
y_predict = model.predict(x_test)
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')
auc = roc_auc_score(y_test, model.predict(x_test, verbose=0), multi_class='ovr')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("The AUC score is:", round(auc, 4))
print("=============================================")

***

#### multi-subnetworks

In [ ]:
class Args_multi:
    # arugments
    epochs=200
    enc_epochs = 50
    bs=32
    enc_bs = 16
    lr=0.0001
    momentum=0.9
    num_classes= 3
    seed=710674

args_multi = Args_multi()

In [ ]:
ori_x = x.copy()

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)
# scaler = RobustScaler()

ori_x[:] = scaler.fit_transform(ori_x[:])
ori_x = ori_x.round(decimals=5)

In [ ]:
########## 3-CLASS CLASSIFICATION ##########
label = y.copy()
label = label.replace({'con': 0})
label = label.replace({'pd': 1})
label = label.replace({'mdd': 2})

ori_y = to_categorical(label, 3) ## into the format of one-hot encoding

In [ ]:
# ########## 3-CLASS CLASSIFICATION ##########
# label = y.copy()
# label = label.replace({'normal': 0})
# label = label.replace({'mild': 1})
# label = label.replace({'mod_severe': 2})

# ori_y = to_categorical(label, 3) ## into the format of one-hot encoding

In [ ]:
data_b1 = ori_x.iloc[:, (ori_x.columns.str.startswith('b1_'))]
data_s = ori_x.iloc[:, (ori_x.columns.str.startswith('s_'))]
data_b2 = ori_x.iloc[:, (ori_x.columns.str.startswith('b2_'))]
data_r = ori_x.iloc[:, (ori_x.columns.str.startswith('r_'))]
data_b3 = ori_x.iloc[:, (ori_x.columns.str.startswith('b3_'))]
data_c = ori_x.iloc[:, (ori_x.columns.str.startswith('c_'))]

In [ ]:
# 입력 정의
inputb1 = Input(shape=(data_b1.shape[1],))
inputs = Input(shape=(data_s.shape[1],))
inputb2 = Input(shape=(data_b2.shape[1],))
inputr = Input(shape=(data_r.shape[1],))
inputb3 = Input(shape=(data_b3.shape[1],))
inputc = Input(shape=(data_c.shape[1],))

* Creating commonly using sub-network

In [ ]:
# 서브 네트워크 정의 함수
def create_subnetwork(input_layer):
    x = Dense(16, activation='relu')(input_layer)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(8, activation='relu')(x)
    return x

In [ ]:
# 서브 네트워크 생성
subnetb1 = create_subnetwork(inputb1)
subnets = create_subnetwork(inputs)
subnetb2 = create_subnetwork(inputb2)
subnetr = create_subnetwork(inputr)
subnetb3 = create_subnetwork(inputb3)
subnetc = create_subnetwork(inputc)

In [ ]:
combined = Concatenate()([subnetb1, subnets, subnetb2, subnetr, subnetb3, subnetc])

In [ ]:
combined

In [ ]:
## adding dense(fc) layers after encoder combining
concat_x = Dense(64, activation='relu')(combined)
concat_x = BatchNormalization()(concat_x)
# concat_x = Dropout(0.5)(concat_x)
concat_x = Dense(32, activation='relu')(concat_x)
concat_x = BatchNormalization()(concat_x)
# concat_x = Dropout(0.5)(concat_x)
concat_x = Dense(16, activation='relu')(concat_x)
concat_x = Dense(8, activation='relu')(concat_x)

# 최종 출력 레이어 정의 (4-class 분류)
final_output = Dense(args_multi.num_classes, activation='softmax')(concat_x)

In [ ]:
b1_train, b1_test, s_train, s_test, b2_train, b2_test, r_train, r_test, b3_train, b3_test, c_train, c_test, y_train, y_test = train_test_split(
    data_b1, data_s, data_b2, data_r, data_b3, data_c, ori_y, test_size=0.2, random_state=710674)

In [ ]:
# LearningRateScheduler 설정
lr_scheduler = LearningRateScheduler(lambda epoch: cyclic_learning_rate(epoch, base_lr=0.01, max_lr=0.006, step_size=200))
# lr_scheduler = LearningRateScheduler(lambda epoch: exp_range_lr(epoch, base_lr=0.01, max_lr=0.006, step_size=300, gamma=0.99994))

In [ ]:
opt = keras.optimizers.SGD(learning_rate = args_multi.lr, decay = 1e-6, momentum = args_multi.momentum)

In [ ]:
# 모델 생성 및 컴파일
model = Model(inputs=[inputb1, inputs, inputb2, inputr, inputb3, inputc], outputs=final_output)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit([b1_train, s_train, b2_train, r_train, b3_train, c_train], y_train, 
          epochs=args_multi.epochs, batch_size=args_multi.bs, callbacks=[lr_scheduler], verbose=2)

In [ ]:
y_predict = model.predict([b1_test, s_test, b2_test, r_test, b3_test, c_test])

y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')
# auc = roc_auc_score(y_test,model.predict([x2_test, x4_test, x5_test, x6_test, x7_test], verbose=0), multi_class='ovr')
auc = roc_auc_score(y_test,model.predict([b1_test, s_test, b2_test, r_test, b3_test, c_test], verbose=0), multi_class='ovr')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("The AUC score is:", round(auc, 4))
print("=============================================")

***

***

## Using VAE/AE as a dimensionality reduction

In [ ]:
x = data_x
y = y

In [ ]:
class Args:
    # arugments
    epochs=200
    enc_epochs = 50
    bs=32
    enc_bs = 16
    lr=0.001
    momentum=0.9
    encoding_dim = 16
    num_classes= 2
    verbose='store_true'
    seed=710674

args = Args()

### Autoencoder for dim reduction

In [ ]:
# input dataset layer
input_layer = Input(shape=(x.shape[1],))

# encoder layers
encoder = Dense(64, activation='relu')(input_layer)
encoder = Dense(32, activation='relu')(encoder)
encoder_out = Dense(args.encoding_dim, activation='relu')(encoder)

# decoder layers
decoder = Dense(32, activation='relu')(encoder_out)
decoder = Dense(64, activation='relu')(decoder)
decoder_out = Dense(x.shape[1], activation='sigmoid')(decoder)

In [ ]:
# AE model
autoencoder = Model(inputs=input_layer, outputs=decoder_out)

# Encoder model (convert input dataset into latent space)
encoder_model = Model(inputs=input_layer, outputs=encoder_out)

# Decoder model (recover latent space/vector into original dataset format)
encoded_input = Input(shape=(args.encoding_dim,))
decoder_layer = autoencoder.layers[-3](encoded_input)
decoder_layer = autoencoder.layers[-2](decoder_layer)
decoder_out = autoencoder.layers[-1](decoder_layer)
decoder_model = Model(inputs=encoded_input, outputs=decoder_layer)

In [ ]:
# Model compile
autoencoder.compile(optimizer='adam', loss='mse')

# Checking each model
autoencoder.summary()
# encoder_model.summary()
# decoder_model.summary()

In [ ]:
## Model training
autoencoder.fit(x, x, epochs = args.enc_epochs, batch_size = args.enc_bs, shuffle=True, validation_split=0.2, verbose=2)

In [ ]:
encoded_data = encoder_model.predict(x)
decoded_data = decoder_model.predict(encoded_data)

In [ ]:
encoded_data.shape

In [ ]:
decoded_data.shape

In [ ]:
y.shape

In [ ]:
decoded_data

In [ ]:
class Args:
    # arugments
    epochs=200
    bs=32
    lr=0.001
    momentum=0.9
    num_classes= 2
    verbose='store_true'
    seed=710674

args = Args()

# np.random.seed(args.seed)
# random.seed(args.seed)
# torch.manual_seed(args.seed)

In [ ]:
x_trainset, x_test, y_trainset, y_test = train_test_split(decoded_data, y, test_size = 0.1, random_state = 710674)
x_train, x_vali, y_train, y_vali = train_test_split(x_trainset, y_trainset, test_size = 0.2, random_state = 710674)

In [ ]:
inputs = np.concatenate((x_train, x_vali), axis = 0)
targets = np.concatenate((y_train, y_vali), axis = 0)

In [ ]:
fold_num = 1
split_num = 5
opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-6, momentum = args.momentum)
kfold = KFold(n_splits = split_num, shuffle = True)
# kfold = StratifiedKFold(n_splits = split_num, shuffle = True)

In [ ]:
############## FOR FOUR-GROUP CLASSIFICATION ###############
class_weight = {0:1, 1: 1.68}

In [ ]:
acc_per_fold = []
loss_per_fold = []

In [ ]:
for train, test in kfold.split(inputs, targets):
    model = Sequential()
    model.add(Dense(128, input_dim = x_train.shape[1], activation = 'relu'))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dropout(0.5)) #drop out
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(args.num_classes, activation = 'softmax'))
    
    ## model compile
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    
    print('----------------------------------------')
    print(f'Training or fold {fold_num} ... ')
    
    ## fit data to model
    history = model.fit(inputs[train], targets[train], batch_size = args.bs, epochs = args.epochs, verbose = 0, class_weight = class_weight)
    
    ## generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test])
    print(f'Score for fold {fold_num}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    ## increasing fold number
    fold_num = fold_num + 1
    
    
    
## Summarizing the results
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'>> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'>>> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'>>> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
y_predict = model.predict(x_test)
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')
auc = roc_auc_score(y_test, model.predict(x_test, verbose=0), multi_class='ovr')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("The AUC score is:", round(auc, 4))
print("=============================================")

## EV input & MV output

> What if we use external variables(ex. demographics) as input variables and set the DNN model to find Medical Variables as an output variables?

### data setup

In [ ]:
data_evinput = data_ori.copy()

In [ ]:
data_evinput['disorder'] = data_evinput['disorder'].replace({"mdd": 0})
data_evinput['disorder'] = data_evinput['disorder'].replace({"pd": 1})
data_evinput['disorder'] = data_evinput['disorder'].replace({"con": 2})

In [ ]:
# data_evinput.head()
# data_evinput.columns

In [ ]:
scale_list = ['HAMD', 'HAMA', 'PDSS', 'ASI', 'APPQ', 'PSWQ', 'SPI', 'PSS', 'BIS', 'SSI']
etc_list = ['sub', 'VISIT', 'disorder_mdd']
demo_list = ['age', 'gender', 'disorder']

In [ ]:
ev_input = data_evinput.loc[:, ('disorder', 'age', 'gender')]
ev_output = data_evinput.drop((scale_list + etc_list + demo_list), axis=1)

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)

ev_input[:] = (scaler.fit_transform(ev_input[:])).round(decimals=6)
ev_output[:] = (scaler.fit_transform(ev_output[:])).round(decimals=6)

### Model

In [ ]:
input_dim = ev_input.shape[1]
output_dim = ev_output.shape[1]

In [ ]:
# 모델 구조 정의
ev_model = Sequential([
    Dense(16, input_dim=input_dim, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'), 
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(output_dim)
])

In [ ]:
# 모델 컴파일
ev_model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 요약 출력
ev_model.summary()

In [ ]:
# 학습 예시 (X_train은 인구학적 변수, Y_train은 생체신호 변수)
ev_model.fit(ev_input, ev_output, epochs=100, batch_size=32, validation_split=0.2)